Setup on new ec2 instance (ubuntu)
1. Install Git
"""
sudo apt update
sudo apt install git
"""

2. Generate ssh key 
"""
ssh-keygen -t rsa -C "your-email@gmail.com"
"""

3. Change to root user and access ssh public key
"""
sudo su
cat ~/.ssh/id_rsa.pub
"""

4. Add public key to GitHub repo (repo > Settings > Deploy Keys > Add key)

5. Create virtual environment
"""
sudo apt install python3.10-venv
python3 -m venv venv
"""

6. Use Jupyter notebook in virtual environment
"""
source venv/bin/activate
pip install ipykernel
python -m ipykernel install --user
"""

7. Install PyTorch
- https://pytorch.org/
- CPU installation
"""
pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
"""

8. Install requirements.txt from YOLOv5 repo
- https://github.com/ultralytics/yolov5/issues/36
"""
pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt
"""

If error with importing cv2 library
"""
sudo apt-get update
sudo apt-get install ffmpeg libsm6 libxext6  -y
sudo apt-get install -y python3-opencv
"""

9. Copy images from local to ec2 instance
"""
scp -i ~/.ssh/dimuto-ai.pem -r /Users/weiching/GitHub/pineapple_carton_crown ubuntu@ec2-46-137-227-33.ap-southeast-1.compute.amazonaws.com:~/product-quality-ai/pineapple_carton_crown
"""

In [1]:
import torch

# Model for inference
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


/home/ubuntu/product-quality-ai/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import cv2
img_path = "../data/raw/pineapple_carton_crown/IMG_20190816_154429.jpg"
img1 = cv2.imread(img_path)
imgs = [img1]

[ WARN:0@0.068] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../data/raw/pineapple_carton_crown/IMG_20190816_154429.jpg'): can't open/read file: check file path/integrity


In [3]:
# Training
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', autoshape=False)  # load pretrained

Using cache found in /home/ubuntu/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-7-28 Python-3.10.4 torch-1.12.0+cpu CPU



In [30]:
# # Split dataset into train, val and test set
# import random
# import os
# import numpy as np

# image_list = [x for x in os.listdir("../data/raw/pineapple_carton_crown/") if x.endswith(".jpg")]
# random.Random(0).shuffle(image_list)
# training_split = 0.8
# len_training = round(len(image_list)*training_split)
# train_images = image_list[:len_training]
# val_images = image_list[len_training:]


# def create_training_dataset(images, type="train"):
#     dir = "../data/raw/pineapple_carton_crown"
#     new_dir = os.path.join(dir, type)

#     if not os.path.exists(new_dir):
#         os.makedirs(new_dir)
#     for i in images:
#         os.rename(os.path.join(dir, i), os.path.join(new_dir, i))
#         os.rename(os.path.join(dir, i.replace(".jpg",".txt")), os.path.join(new_dir, i.replace(".jpg",".txt")))

# # create_training_dataset(train_images, type="train")
# # create_training_dataset(val_images, type="val")

In [5]:
import os
os.chdir("../yolov5")

In [10]:
!python yolov5/train.py --img 416 --batch 16 --epochs 10 --data ../datasets/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

train: weights=, cfg=./models/custom_yolov5s.yaml, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
Traceback (most recent call last):
  File "/home/ubuntu/product-quality-ai/yolov5/train.py", line 642, in <module>
    main(opt)
  File "/home/ubuntu/product-quality-ai/yolov5/train.py", line 512, in main
    check_file(opt.data), check_yaml(opt.cfg), check_yaml(opt.hyp), str(opt.weights), str(opt.project)  # checks
  Fi

In [ ]:
# YOLOV6
!python tools/train.py --batch 32 --conf configs/yolov6s.py --data data/data.yml --device cpu

# YOLOV5
!python train.py --batch 3 --epochs 30 --data data/data.yml --weights yolov5s.pt --device cpu